### Yield From - Throwing Exceptions

In [1]:
class CloseCoroutine(Exception):
    pass

In [7]:
def echo():
    try:
        while True:
            received = yield
            print(received)
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called/or GeneratorExit thrown')
        return 'from a GeneratorExit'

In [3]:
e = echo()
next(e)

In [8]:
e.throw(CloseCoroutine)

CloseCoroutine: 

In [9]:
e = echo()
next(e)
e.close()

closed method was called/or GeneratorExit thrown


In [10]:
e = echo()
next(e)
e.throw(GeneratorExit)

closed method was called/or GeneratorExit thrown


StopIteration: from a GeneratorExit

In [11]:
def delegator():
    result = yield from echo()
    yield 'subgen closed and returned:', result
    print('delegator closing...')

In [12]:
d = delegator()
next(d)
d.send('hello')

hello


In [13]:
d.throw(CloseCoroutine)

('subgen closed and returned:', 'coro was closed')

In [14]:
class IgnoreMe(Exception):
    pass

In [16]:
def echo():
    try:
        while True:
            try:
                received = yield
                print(received)
            except IgnoreMe:
                yield "I'm ignoring you!"
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called/or GeneratoRexit thrown!')
        return 'from a GeneratorExit'

In [17]:
d = delegator()
next(d)

In [18]:
d.send('python')

python


In [19]:
result = d.throw(IgnoreMe, 1000)

/tmp/ipykernel_783/390020266.py:1: DeprecationWarning: the (type, exc, tb) signature of throw() is deprecated, use the single-arg signature instead.
  result = d.throw(IgnoreMe, 1000)


In [20]:
result

"I'm ignoring you!"

In [21]:
d.send('rocks!')

In [22]:
d.send('rocks!')

rocks!


In [23]:
d.close()

closed method was called/or GeneratoRexit thrown!


In [24]:
def echo():
    output = None
    try:
        while True:
            try:
                received = yield output
                print(received)
            except IgnoreMe:
                output = "I'm ignoring you!"
            else:
                output = None
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called/or GeneratoRexit thrown!')
        return 'from a GeneratorExit'

In [25]:
d = delegator()
next(d)

In [26]:
d.send('python')

python


In [28]:
result = d.throw(IgnoreMe)

In [29]:
result

"I'm ignoring you!"

In [32]:
result = d.send('rocks')

rocks


In [35]:
type(result)

NoneType

In [36]:
d.close()

closed method was called/or GeneratoRexit thrown!


In [37]:
def echo():
    while True:
        received = yield
        print(received)

In [38]:
def delegator():
    yield from echo()

In [39]:
d = delegator()
next(d)

In [40]:
d.throw(ValueError)

ValueError: 

In [41]:
def delegator():
    try:
        yield from echo()
    except ValueError:
        print('delegator got the value error')

In [42]:
d = delegator()
next(d)
d.throw(ValueError)

delegator got the value error


StopIteration: 

In [44]:
def delegator():
    while True:
        try:
            yield from echo()
        except ValueError:
            print('delegator got the value error')

In [45]:
d = delegator()
d.__next__()

In [46]:
d.send('python')

python


In [47]:
d.throw(ValueError)

delegator got the value error


In [48]:
d.send('rocks!')

rocks!


In [49]:
d.throw(IgnoreMe)

IgnoreMe: 

In [50]:
class WriteAverage(Exception):
    pass

In [51]:
def averager(out_file):
    total = 0
    count = 0
    average = None
    with open(out_file, 'w') as f:
        f.write('count,average\n')
        while True:
            try:
                received = yield average
                total += received
                count += 1
                average = total / count
            except WriteAverage:
                if average is not None:
                    print('saving average to file:', average)
                    f.write(f'{count}, {average}\n')

In [52]:
avg = averager('sample.csv')
next(avg)

In [53]:
avg.send(1)
avg.send(2)

1.5

In [54]:
avg.throw(WriteAverage)

saving average to file: 1.5


1.5

In [55]:
avg.send(3)

2.0

In [56]:
avg.send(2)

2.0

In [57]:
avg.throw(WriteAverage)

saving average to file: 2.0


2.0

In [58]:
avg.close()

In [59]:
with open('sample.csv') as f:
    for row in f:
        print(row.strip())

count,average
2, 1.5
4, 2.0


In [60]:
def delegator():
    yield from averager('sample.csv')

In [61]:
d = delegator()
next(d)

In [62]:
d.send(1)

1.0

In [63]:
d.send(2)

1.5

In [64]:
d.send(3)

2.0

In [65]:
d.send(4)

2.5

In [66]:
d.throw(WriteAverage)

saving average to file: 2.5


2.5

In [67]:
d.send(5)

3.0

In [68]:
d.throw(WriteAverage)

saving average to file: 3.0


3.0

In [70]:
d.close()

In [71]:
with open('sample.csv') as f:
    for row in f:
        print(row.strip())

count,average
4, 2.5
5, 3.0
